## TF-IDF schema0

### tf, df, dictionary

In [1]:
from collections import Counter

def dic_(allDocument): # alldocument=collection: list
    long_str = " ".join(allDocument)
    b=list(set(list(long_str.split())))
    return sorted(b)

def tf_(doc_que):
    counts = Counter(list(doc_que.split()))
    return dict(counts)

def df_(term, allDocuments):  # df: no. of occurance of a term in whole collection
    dic=dict.fromkeys(term, 0)
    for i in allDocuments:
        n=0
        for word in term:
            if word in i.split():         
                dic[word]=dic[word]+1
            n+=1  
    return dic  

### tf-idf

In [2]:
import math

def tfidf_(query_s,c,df,allDocument): # query_s: Series (df['col']), allDocument:list

    all_tfidf=[]
    for query in query_s:
        q_tf=tf_(query)
        max_tf=max(q_tf.values())
        tfidf=[]
        for word in c:
            if word not in q_tf: # query item does not shown in doc collection
                    value=0
            else:
                value=(1+math.log10(q_tf.get(word)))/(1+math.log10(max_tf))*math.log10(len(allDocument)/df.get(word))
            tfidf.append(value)
        all_tfidf.append(tfidf)
    return np.array(all_tfidf) #vector

### cosine similarity

In [3]:
import numpy as np
def cosine_similarity(a,b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

### Random Projection_tfidf

In [4]:
import pandas as pd
import numpy as np

df_doc=pd.read_csv('d_tfidf2000.csv')
df_query=pd.read_csv('q_tfidf2000.csv')

In [5]:
df_doc=df_doc.drop(['Unnamed: 0'],axis=1)
df_query=df_query.drop(['Unnamed: 0'],axis=1)

In [6]:
d_tfidf=df_doc.as_matrix(columns=None)
q_tfidf=df_query.as_matrix(columns=None)

In [7]:
df_query_origin=pd.read_excel('df_query_pre.xlsx')
df_query=pd.merge(df_query,df_query_origin[['Query id']], left_index=True, right_index=True,how='left')
df_doc_origin=pd.read_excel('df_doc_pre.xlsx')
df_doc=pd.merge(df_doc,df_doc_origin[['Doc id']], left_index=True, right_index=True,how='left')

### k-means

In [14]:
import numpy as np

def initialize_centroids(points, k, ran_seed):
    np.random.seed(seed=ran_seed)
    centroids = points.copy()
    np.random.shuffle(centroids)
    return centroids[:k]

def closest_centroid(points, centroids):
    cluster_label=[]
    for doc in range(len(points)):
        similarity=[]
        for l in range(len(centroids)):
            sim=cosine_similarity(points[doc],centroids[l])
            similarity.append(sim)
        cluster_label.append(np.argmax(similarity))
    return np.asarray(cluster_label)

def move_centroids(points, closest, centroids):
    mean=[]
    for k in range(centroids.shape[0]):
        if len(d_tfidf[closest==k])==0:
            mean.append([0.5])
        else:
            mean.append(d_tfidf[closest==k].mean(axis=0))
        move_centroid= np.zeros([len(mean),len(max(mean,key = lambda x: len(x)))])
        for i,j in enumerate(mean):
            move_centroid[i][0:len(j)] = j
    return move_centroid
    #return np.array([points[closest==k].mean(axis=0) for k in range(centroids.shape[0])])
    
def k_means(points, k,ran_seed):
    leaders=initialize_centroids(points, k, ran_seed)
    move=True
    while move != False:
        leaders_new=move_centroids(points, closest_centroid(points, leaders), leaders)
        for i in range(len(leaders)):
            if cosine_similarity(leaders[i],leaders_new[i])>0.99999:
                move=False
            else:
                move=True
        leaders=leaders_new
    return leaders, closest_centroid(points, leaders) # cluster label

def pre_clustering(doc,k,ran_seed):
    cluster=k_means(doc, k, ran_seed)
    leader=cluster[0]
    cluster_label=cluster[1]
    df_d_tfidf = pd.DataFrame(doc)
    df_d_tfidf['label']=cluster_label
    return leader, df_d_tfidf

### Implementation: Cosine Similarity

In [15]:
import math
import operator
import time

def cosSimilarityImp(q_tfidf,leader, df_d_tfidf):
    
    start = time.time()
    sim_all={}
    for q in range(len(q_tfidf)):
        leader_sim={}
        sim={}
        sim_all[q]=sim

        for l in range(len(leader)):
            leader_sim[l]=cosine_similarity(leader[l],q_tfidf[q])

        closest_cluster_label=max(leader_sim.items(), key=operator.itemgetter(1))[0]
        closest_cluster=df_d_tfidf.loc[df_d_tfidf['label'] == closest_cluster_label].drop(['label'],axis=1)

        for d in closest_cluster.index:
            sim[d]=cosine_similarity(closest_cluster.loc[d],q_tfidf[q])

    end = time.time()
    print('similarity time: ',end - start)

    df_similarity= pd.DataFrame(data=sim_all)
    df_similarity['d_index']=df_similarity.index
    df1 = df_similarity.reset_index(drop=True)
    df2 = pd.melt(df1, id_vars=["d_index"], var_name="q_index", value_name="similarity")
    df2=df2.sort_values(by=['q_index','similarity'],ascending=[True,False])

    df_query['q_index']=df_query.index
    df_doc['d_index']=df_doc.index
    df2=pd.merge(df2,df_query[['Query id','q_index']],on='q_index',how='left')
    df2=pd.merge(df2,df_doc[['Doc id','d_index']],on='d_index',how='left')
    df_similarity_final=df2[['Query id', 'Doc id', 'similarity']]
    df_similarity_final=df_similarity_final.dropna()

    # measuremnt format
    rel_label = pd.read_excel("all2-1-0.qrel.xlsx")
    rel_similarity=pd.merge(df_similarity_final[['Query id','Doc id']],rel_label, on=['Query id','Doc id'], how='left')
    rel_similarity=rel_similarity.fillna(value=0)
    rel_similarity=rel_similarity.drop(['Doc id'],axis=1)
    s = rel_similarity.groupby('Query id')['Rel_level'].apply(lambda x: x.tolist())
    correct_input=s.values

    return correct_input

#### MAP

In [16]:
import numpy as np 
# presicion 
def precision(y_true):
#y_true: this list reordered y_true list
    p=[]
    a=0
    for i in range(len(y_true)):
        if y_true[i]==1:
            a+=1
            precision=a/(i+1)
            p.append(precision)
    #print(p)
    return p

# average precision
def AP(y_true):
    p=precision(y_true)
    if len(p)!=0:
        AP=sum(p)/len(p)
    else:
        AP=0
    #print(AP)
    return AP


def MAP(list_true):# query list 
    ap=[]
    for i in range(len(list_true)):
        ap.append(AP(list_true[i]))
    total=sum(ap)
    #print(ap)
    return total/(len(list_true))

#### nDCG1

In [17]:
import numpy as np 
def ndcg1(correct):
    if sum(correct)==0:
        return 0
    else:
        dcg=0
        for i in range(len(correct)):
            gain=2**correct[i]-1
            discounts=np.log2(i+2)
            dcg=dcg+gain/discounts
    
        #idcg
        order = np.argsort(correct)
        sort_correct = np.take(correct, order[::-1])
        idcg=0
        for i in range(len(sort_correct)):
            gain=2**sort_correct[i]-1
            discounts=np.log2(i+2)
            idcg=idcg+gain/discounts
        #print(dcg)
        #print(idcg)
        return dcg/idcg

def mean_ndcg1(correct_list):
    b=0
    for correct in correct_list:
        a=ndcg1(correct)
        b=b+a
    return b/len(correct_list)

### Performance

In [21]:
leader_size=[300]
for i in leader_size:
    print('leader size:',i)
    cluster=pre_clustering(d_tfidf,i,0)
    result=cosSimilarityImp(q_tfidf,cluster[0], cluster[1])
    print('MAP:',MAP(result)) 
    print('nDCG:',mean_ndcg1(result))
    print()

leader size: 300
similarity time:  95.25399971008301
MAP: 0.08799739678910269
nDCG: 0.153992917227

